In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
project_dir = '/Users/volodymyr.kotov/Desktop/Kaggle/nfl_big_data_bowl_2025/data/nfl-big-data-bowl-2025'

In [3]:
games = pd.read_csv(f'{project_dir}/games.csv')
plays = pd.read_csv(f'{project_dir}/plays.csv')
players = pd.read_csv(f'{project_dir}/players.csv')

In [86]:
week = 9
tracking_df = pd.read_csv(f'{project_dir}/tracking_week_{week}.csv')
tracking_df['tracking_week'] = week

In [87]:
def get_speed(distance: float, time, lead_event_time) -> None:
    """gets speed in yards/second"""
    if not pd.isna(lead_event_time) and not pd.isna(time):
        time_delta = (lead_event_time - time).total_seconds()
        if time_delta > 0:
            return distance / time_delta
    return None

def get_distance(x: float, x_pass_landed: float, y: float, y_pass_landed: float) -> None:
    """gets distance in yards"""
    return np.pow(
            (
                np.pow((x - x_pass_landed), 2) + 
                np.pow((y - y_pass_landed), 2)
            ),
        0.5
    )

In [88]:
tracking_df.shape[0]

5671685

In [89]:
tracking_df_extended = tracking_df.copy()
tracking_df_extended['uid'] = tracking_df_extended.index

tracking_df_pass_events = tracking_df_extended[
    (tracking_df_extended['event'].isin(
                [
                    'pass_forward', 
                    'pass_arrived', 
                    'pass_outcome_incomplete', 
                    'pass_outcome_caught', 
                    'pass_outcome_interception', 
                    'qb_strip_sack'
                ]
            )
        ) & 
    (tracking_df_extended['displayName'] == 'football')
]

tracking_df_pass_events['lead_event'] = tracking_df_pass_events.groupby(['gameId', 'playId'])['event'].shift(-1)
tracking_df_pass_events['lead_event_time'] = tracking_df_pass_events.groupby(['gameId', 'playId'])['time'].shift(-1)
tracking_df_pass_events['x_pass_landed'] = tracking_df_pass_events.groupby(['gameId', 'playId'])['x'].shift(-1)
tracking_df_pass_events['y_pass_landed'] = tracking_df_pass_events.groupby(['gameId', 'playId'])['y'].shift(-1)

/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_22054/1137521624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracking_df_pass_events['lead_event'] = tracking_df_pass_events.groupby(['gameId', 'playId'])['event'].shift(-1)
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_22054/1137521624.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracking_df_pass_events['lead_event_time'] = tracking_df_pass_events.groupby(['gameId', 'playId'])['time'].shift(-1)
/var/folders/mv/18wf18mn

In [90]:
#currently it will produce only a join to football player (a ball) and will show the event type
tracking_df_extended = pd.merge(
    tracking_df_extended,
    tracking_df_pass_events[['uid', 'lead_event', 'lead_event_time', 'x_pass_landed', 'y_pass_landed']],
    on='uid',
    how='left'
)

tracking_df_extended = pd.merge(
    tracking_df_extended,
    players,
    on=['nflId', 'displayName'],
    how='left'
)


#add a column is_qb  
tracking_df_extended['is_qb'] = np.where(
    tracking_df_extended['position'] == 'QB', True, False
)

In [91]:
tracking_df_extended['distance'] = tracking_df_extended[['x', 'x_pass_landed', 'y', 'y_pass_landed']].apply(
    lambda row: get_distance(row['x'], row['x_pass_landed'], row['y'], row['y_pass_landed']), 
    axis=1
)

In [92]:
tracking_df_extended['time'] = pd.to_datetime(tracking_df_extended['time'], format='mixed')
tracking_df_extended['lead_event_time'] = pd.to_datetime(tracking_df_extended['lead_event_time'], format='mixed')

In [93]:
tracking_df_extended['speed'] = tracking_df_extended.apply(
    lambda row: get_speed(row['distance'], row['time'], row['lead_event_time']), 
    axis=1
)

In [94]:
tracking_df_extended.shape[0]

5671685

In [95]:
tracking_df_extended.to_csv(f'tracking_week_{week}_extended.csv', index=False)

In [99]:
project_dir = '/Users/volodymyr.kotov/Desktop/Kaggle/nfl_big_data_bowl_2025'

In [100]:
final_df = pd.DataFrame()
for i in range(1, 10):
    temp_df = pd.read_csv(f'{project_dir}/tracking_week_{week}_extended.csv')
    final_df = pd.concat([final_df, temp_df])

/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_22054/356917563.py:3: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'{project_dir}/tracking_week_{week}_extended.csv')
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_22054/356917563.py:3: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'{project_dir}/tracking_week_{week}_extended.csv')
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_22054/356917563.py:3: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'{project_dir}/tracking_week_{week}_extended.csv')
/var/folders/mv/18wf18mn1c9bkwrp3fxhqr9c0000gp/T/ipykernel_22054/356917563.py:3: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  temp_df = pd.read_csv(f'{proje

In [101]:
final_df.to_csv('tracking_weeks.csv', index=False)

In [ ]:
git statu